In [2]:
# import necessary libraries

import requests
from bs4 import BeautifulSoup
import csv
import time
import http.client

In [ ]:
# initialise global variables to store the scraped information

tid = []               # contains all the thread IDs 
messages = []          # stores all the message contents
subjects = []          # stores the subjects of the messages

In [ ]:
# function to connect to the given forum ID, retrieve all the threads present within that forum and store it in tid

def connect(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    }
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    
    soup = BeautifulSoup(res.text, 'html.parser')
    soup.prettify()

    # all threadIDs have the following id
    
    hrefs = soup.find_all("a", {"id": "jive-thread-0"})

    for href in hrefs:
        href = str(href)
        href = href.split('href="')[1].split("&")[0]
        # href of form threads.jspa?threadID=xxx
        tid.append(href)
    return tid

In [ ]:
# for each thread ID, fetches the messages present within thread

def fetch(url, fname):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    }
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    
    soup = BeautifulSoup(res.text, 'html.parser')
    soup.prettify()
    
    # all messages have the following subject class
    
    subs = soup.find_all("span", {"class": "jive-subject"})
    
    for line in subs:
        subjects.append(line.text.strip())
    
    # all messages have the following body class
    
    msgs = soup.find_all("div", {"class": "jive-message-body"})
    
    for line in msgs:
        msg = line.text.strip()
        messages.append(msg)
        
        with open(fname + '.tsv', 'w', encoding='utf-8') as tsvfile:
            writer = csv.writer(tsvfile, delimiter='\t')
            writer.writerow(["label", "description"])
            for i, j in zip(subjects, messages):
                writer.writerow([fname, i + "\n" + j])

In [ ]:
# code snippet to fetch all the threadIDs for given service

id = 60                    # target forumID
limit = 10268                 # total number of threads
pname = "Amazon RDS"              # target product name

i = 0

while i <= limit:
    if i >= 33300:
        url = "https://forums.aws.amazon.com/forum.jspa?forumID="+str(id)+"&start="+str(i)
        tid = connect(url)
        print("Fetched " + str(i) + " threads")
    i = i + 25
    

In [ ]:
print(tid)

In [ ]:
# code snippet to fetch all the message for the corresponding threadID and store in tsv

i = 0

for thread in tid:
    try:
        print("Processing thread #" + str(i))
        fetch("https://forums.aws.amazon.com/" + thread, pname)
    except:
        time.sleep(5)
        i = i - 1
    except http.client.HTTPException as e:
        time.sleep(5)
        i = i - 1
    i += 1